In [ ]:
"""
LangGraph Persistence Tutorial - State Management & Time Travel

This notebook demonstrates how to build persistent workflows with LangGraph that can:

1. Save and restore workflow state across executions
2. Maintain separate conversation threads (sessions)
3. Access complete history of state changes at each step
4. "Time travel" - jump back to any previous checkpoint and continue
5. Manually update past states to create alternate execution paths

Key Features Covered:
- MemorySaver checkpointer for state persistence
- Thread-based session management (each thread_id = separate conversation)
- State history tracking and retrieval
- Checkpoint-based resumption from any point in workflow
- Manual state updates for advanced control

Use Cases: Multi-turn conversations, debugging workflows, A/B testing different paths,
creating branching narratives, and maintaining user sessions across app restarts.
"""

# Import libraries for building a persistent workflow
# MemorySaver allows us to save and restore workflow state
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
# Set up the language model for generating jokes and explanations
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv(dotenv_path='../.env')

model = ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
# Define the state structure for our joke workflow
# This tracks the topic, generated joke, and explanation
class JokeState(TypedDict):
    topic : str
    joke : str
    explanation : str

In [ ]:
# Function to generate a joke based on the given topic
def generate_joke(state: JokeState):
    prompt = f"Generate a joke on the topic : {state['topic']}"
    response = model.invoke(prompt).content
    
    return {'joke' : response}

# Function to explain why the joke is funny
def generate_explanation(state: JokeState):
    prompt = f"Write an explanation for the joke - {state['joke']}"
    response = model.invoke(prompt).content
    
    return {'explanation' : response}

In [ ]:
# Build the workflow graph with persistence enabled
# The checkpointer saves state at each step so we can resume or review history
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation' , generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

# Add memory saver to enable persistence
checkpointer = MemorySaver()

workflow = graph.compile(checkpointer= checkpointer)

In [ ]:
# Run the workflow for the first time with thread_id "1"
# Each thread_id creates a separate conversation/session
config1 = {"configurable" : {"thread_id" : "1"}}
workflow.invoke({'topic' : 'Donald Trump'}, config=config1)

{'topic': 'Donald Trump',
 'joke': 'Why did Donald Trump bring a ladder to the White House?\n\nBecause he wanted to take his wall to the next level! (get it?)',
 'explanation': 'A clever play on words. This joke relies on a double meaning of the phrase "take it to the next level." In a literal sense, a ladder is used to reach higher physical levels or heights. However, the phrase "take it to the next level" is also an idiomatic expression meaning to improve, enhance, or escalate something.\n\nIn this joke, the punchline "take his wall to the next level" is a wordplay referencing Donald Trump\'s controversial proposal to build a wall along the US-Mexico border. By bringing a ladder to the White House, Trump is literally trying to take his wall to a higher physical level, but the phrase also implies that he\'s trying to improve or escalate his wall project.\n\nThe humor comes from the unexpected twist on the phrase\'s meaning, creating a clever and amusing connection between the setup (T

In [ ]:
# Get the current/final state of thread 1
# This shows the complete state after the workflow finished
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Donald Trump', 'joke': 'Why did Donald Trump bring a ladder to the White House?\n\nBecause he wanted to take his wall to the next level! (get it?)', 'explanation': 'A clever play on words. This joke relies on a double meaning of the phrase "take it to the next level." In a literal sense, a ladder is used to reach higher physical levels or heights. However, the phrase "take it to the next level" is also an idiomatic expression meaning to improve, enhance, or escalate something.\n\nIn this joke, the punchline "take his wall to the next level" is a wordplay referencing Donald Trump\'s controversial proposal to build a wall along the US-Mexico border. By bringing a ladder to the White House, Trump is literally trying to take his wall to a higher physical level, but the phrase also implies that he\'s trying to improve or escalate his wall project.\n\nThe humor comes from the unexpected twist on the phrase\'s meaning, creating a clever and amusing connection b

In [ ]:
# View the complete history of state changes for thread 1
# This shows how the state evolved at each step of the workflow
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Donald Trump', 'joke': 'Why did Donald Trump bring a ladder to the White House?\n\nBecause he wanted to take his wall to the next level! (get it?)', 'explanation': 'A clever play on words. This joke relies on a double meaning of the phrase "take it to the next level." In a literal sense, a ladder is used to reach higher physical levels or heights. However, the phrase "take it to the next level" is also an idiomatic expression meaning to improve, enhance, or escalate something.\n\nIn this joke, the punchline "take his wall to the next level" is a wordplay referencing Donald Trump\'s controversial proposal to build a wall along the US-Mexico border. By bringing a ladder to the White House, Trump is literally trying to take his wall to a higher physical level, but the phrase also implies that he\'s trying to improve or escalate his wall project.\n\nThe humor comes from the unexpected twist on the phrase\'s meaning, creating a clever and amusing connection 

In [ ]:
# Start a new conversation thread with a different topic
# Thread 2 is completely separate from thread 1
config2 = {"configurable" : {"thread_id" : "2"}}
workflow.invoke({'topic' : 'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship!',
 'explanation': 'A classic play on words. This joke is funny because it uses a clever pun to create a humorous connection between the setup and the punchline. \n\nThe joke starts by asking why the spaghetti refused to get married, which sets up the expectation that the reason will be related to relationships or commitment. The punchline subverts this expectation by using the phrase "tangled up" in a literal and figurative sense. \n\nSpaghetti is a type of long, thin, cylindrical pasta that can easily become entangled or "tangled up" when not cooked or stored properly. However, the phrase "tangled up" is also an idiomatic expression that means to become deeply involved or embroiled in a complicated situation, often in the context of relationships.\n\nThe joke relies on this double meaning of "tangled up" to create a wordplay that connects the phys

In [ ]:
# Check the history of thread 2 - notice it's independent of thread 1
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship!', 'explanation': 'A classic play on words. This joke is funny because it uses a clever pun to create a humorous connection between the setup and the punchline. \n\nThe joke starts by asking why the spaghetti refused to get married, which sets up the expectation that the reason will be related to relationships or commitment. The punchline subverts this expectation by using the phrase "tangled up" in a literal and figurative sense. \n\nSpaghetti is a type of long, thin, cylindrical pasta that can easily become entangled or "tangled up" when not cooked or stored properly. However, the phrase "tangled up" is also an idiomatic expression that means to become deeply involved or embroiled in a complicated situation, often in the context of relationships.\n\nThe joke relies on this double meaning of "tangled up" to create a wordplay th

# Time Travel - Go Back to Previous States
We can jump back to any previous checkpoint in our workflow history and continue from there.

In [ ]:
# Access a specific checkpoint from thread 2's history
# The checkpoint_id lets us jump to any point in the workflow
workflow.get_state({"configurable" : {"thread_id": 2, "checkpoint_id" : "1f070350-40a5-6bd8-8000-94945e440955"}})

StateSnapshot(values={'topic': 'pasta'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f070350-40a5-6bd8-8000-94945e440955'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-03T06:42:25.185071+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f070350-40a3-6064-bfff-360276f26118'}}, tasks=(PregelTask(id='ad376966-fbb9-c1ac-52ce-22c8ce04d5a8', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship!'}),), interrupts=())

In [ ]:
# Resume the workflow from that specific checkpoint
# This continues execution from where we left off at that point
workflow.invoke(None, {"configurable" : {"thread_id": 2, "checkpoint_id" : "1f070350-40a5-6bd8-8000-94945e440955"}})

{'topic': 'pasta',
 'joke': 'Why did the pasta go to therapy?\n\nBecause it was feeling a little "drained" and wanted to work through some "saucy" issues!',
 'explanation': 'A clever play on words. This joke relies on a form of wordplay called a pun, which involves using a word or phrase that has multiple meanings or sounds similar to another word.\n\nIn this joke, the setup "Why did the pasta go to therapy?" primes the listener to expect a reason related to emotional or psychological issues. The punchline "it was feeling a little \'drained\' and wanted to work through some \'saucy\' issues" uses two key phrases to create the humor:\n\n1. "Feeling a little \'drained\'": This phrase has a double meaning. In a literal sense, pasta is often drained of water after cooking, so the word "drained" is associated with the physical process of cooking pasta. However, in an emotional context, "feeling drained" is a common idiomatic expression for feeling exhausted, empty, or depleted. The joke exp

In [ ]:
# See how the history has changed after resuming from the checkpoint
list(workflow.get_state_history(config=config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta go to therapy?\n\nBecause it was feeling a little "drained" and wanted to work through some "saucy" issues!', 'explanation': 'A clever play on words. This joke relies on a form of wordplay called a pun, which involves using a word or phrase that has multiple meanings or sounds similar to another word.\n\nIn this joke, the setup "Why did the pasta go to therapy?" primes the listener to expect a reason related to emotional or psychological issues. The punchline "it was feeling a little \'drained\' and wanted to work through some \'saucy\' issues" uses two key phrases to create the humor:\n\n1. "Feeling a little \'drained\'": This phrase has a double meaning. In a literal sense, pasta is often drained of water after cooking, so the word "drained" is associated with the physical process of cooking pasta. However, in an emotional context, "feeling drained" is a common idiomatic expression for feeling exhausted, empty, or de

In [ ]:
# Manually update the state at a specific checkpoint
# This changes the topic from 'pasta' to 'samosa' at that point in history
workflow.update_state({"configurable": {"thread_id": "2", "checkpoint_id" : "1f070350-40a5-6bd8-8000-94945e440955", "checkpoint_ns":""}}, {'topic': 'samosa'})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f07037e-4978-60e8-8001-388de4fa4c0c'}}

In [ ]:
# Check the history after manually updating the state
# Notice how the topic has changed in the workflow history
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f07037e-4978-60e8-8001-388de4fa4c0c'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-03T07:03:00.913175+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f070350-40a5-6bd8-8000-94945e440955'}}, tasks=(PregelTask(id='fe0c0558-1b5e-f8ed-ad29-452dd7455390', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship! (get it?)', 'explanation': 'A clever play on words. The joke relies on a double meaning of the phrase "tangled up" to create the punchline. \n\nIn a literal sense, spaghetti is a long, thin, and flexible type of pasta t

In [ ]:
# Continue the workflow from the updated checkpoint
# This will generate a new joke with the updated topic 'samosa'
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f07037e-4978-60e8-8001-388de4fa4c0c"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling a little "crunchy" under the pressure and had a lot of "filling" emotional issues to work through! (get it?)',
 'explanation': 'A deliciously clever joke. Let\'s break it down:\n\nThe joke starts by setting up a unexpected scenario: a samosa, a type of fried or baked pastry, going to therapy. This already piques the listener\'s interest, as it\'s unusual to think of an inanimate object, especially a food item, seeking therapy.\n\nThe punchline is where the wordplay comes in. The joke relies on two key phrases: "crunchy" and "filling." \n\n* "Crunchy" has a double meaning here. In one sense, a samosa is a crunchy food item, known for its crispy exterior. However, "feeling crunchy" is also a play on words, as it sounds similar to "feeling crunched," which means feeling overwhelmed or stressed by pressure.\n* "Filling" is another clever play on words. A samosa typically has a filling, such as spiced 

In [ ]:
# Final check of the complete history showing all our time travel adventures
# You can see the original pasta joke, the manual update to samosa, and the new execution
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling a little "crunchy" under the pressure and had a lot of "filling" emotional issues to work through! (get it?)', 'explanation': 'A deliciously clever joke. Let\'s break it down:\n\nThe joke starts by setting up a unexpected scenario: a samosa, a type of fried or baked pastry, going to therapy. This already piques the listener\'s interest, as it\'s unusual to think of an inanimate object, especially a food item, seeking therapy.\n\nThe punchline is where the wordplay comes in. The joke relies on two key phrases: "crunchy" and "filling." \n\n* "Crunchy" has a double meaning here. In one sense, a samosa is a crunchy food item, known for its crispy exterior. However, "feeling crunchy" is also a play on words, as it sounds similar to "feeling crunched," which means feeling overwhelmed or stressed by pressure.\n* "Filling" is another clever play on words. A samosa typically has a fill